<a href="https://colab.research.google.com/github/surajbhala/GenAI/blob/main/QA_Rag_System_using_Wikipedia_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/32

In [2]:
from getpass import getpass
OPENAI_KEY = getpass('Enter key')

Enter key··········


In [3]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_KEY


In [4]:
from langchain_openai import OpenAIEmbeddings

In [5]:
openai_embed_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

### Chroma Vector DB

In [6]:
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

Downloading...
From (original): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW
From (redirected): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW&confirm=t&uuid=771113ff-160b-4e82-828d-682a75d32bb1
To: /content/simplewiki-2020-11-01.jsonl.gz
100% 50.2M/50.2M [00:00<00:00, 76.1MB/s]


In [8]:
import gzip
import json
wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath,'rt',encoding='utf-8') as fin:
  for line in fin:
    data = json.loads(line.strip())

    docs.append({
        'metadata' :
                    {
                        'title':data.get('title'),
                        'aritcle_id':data.get('id')
                    },
        'data': ''.join(data.get('paragraphs')[0:3])
    })
print("Passages: ",len(docs))

Passages:  169597


In [9]:
docs = [doc for doc in docs for x in ['linguistics','india','cheetah']
              if x in doc['data'].lower().split()]



In [10]:
len(docs)

1342

In [11]:
docs[:3]

[{'metadata': {'title': 'Kurgan hypothesis', 'aritcle_id': '72554'},
  'data': 'The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language.It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion.The Kurgan model is the most widely accepted theory on the origins of Indo-European.'},
 {'metadata': {'title': 'Marija Gimbutas', 'aritcle_id': '72558'},
  'data': 'Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'},
 {'metadata': {'title': 'Basil', 'aritcle_id': '73985'},
  'data': 'Basil ("Ocimum basilicum

### Create Langchain documents

In [12]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc['data'],
                 metadata = doc['metadata']) for doc in docs]

In [13]:
docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'aritcle_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language.It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion.The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'aritcle_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'aritcle_id': '73985'}, page_content='

In [14]:
len(docs)

1342

### Split larger documents into smaller chunks

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 2000,chunk_overlap = 300)
chunked_docs = splitter.split_documents(docs)

In [16]:
chunked_docs[:3]

[Document(metadata={'title': 'Kurgan hypothesis', 'aritcle_id': '72554'}, page_content='The Kurgan model of Indo-European origins is about both the people and their Proto-Indo-European language.It uses both archaeology and linguistics to show the history of their culture at different stages of the Indo-European expansion.The Kurgan model is the most widely accepted theory on the origins of Indo-European.'),
 Document(metadata={'title': 'Marija Gimbutas', 'aritcle_id': '72558'}, page_content='Marija Gimbutas (Lithuanian: Marija Gimbutienė, born Marija Birutė Alseikaitė) (Vilnius, Lithuania, January 23, 1921 – Los Angeles, United States February 2, 1994), was a Lithuanian-American archeologist, known for her research into the Neolithic and Bronze Age cultures of "Old Europe" and the theories that she introduced. Between 1946 and 1971, her writings merged traditional spadework with linguistics and mythologies.'),
 Document(metadata={'title': 'Basil', 'aritcle_id': '73985'}, page_content='

In [17]:
len(chunked_docs)

1366

### Create a vector DB and persist on disk

In [23]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 6.1 MB/s eta 0:00

In [24]:
from langchain_chroma import Chroma

chroma_db = Chroma.from_documents(
    documents = chunked_docs,
    embedding = openai_embed_model,
    persist_directory = 'chroma_db',
    collection_metadata = {'hnsw:space':'cosine'}
)

In [25]:
from langchain_openai import ChatOpenAI
chatgpt = ChatOpenAI(model = 'gpt-3.5-turbo',temperature=0)

### Chained Retrieval Pipeline

In [26]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever

In [27]:
#REtriever 1 -- simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type = "similarity",
                                              searc_kwargs={"k":5})
## decides which of the initially retrieved documents to filter out and which ones to return

_filter = LLMChainFilter.from_llm(llm = chatgpt)

In [28]:
## Retriever 2 - retrieves the documents similar to query and then applies the filter
compressor_retriever = ContextualCompressionRetriever(
    base_compressor = _filter,
    base_retriever = similarity_retriever
)

In [29]:
##download an open-source reranker model - BAAI/bge-reranker-v2-m3

In [31]:
reranker = HuggingFaceCrossEncoder(model_name = "BAAI/bge-reranker-large")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [38]:
reranker_compressor = CrossEncoderReranker(model = reranker,top_n = 1)

In [39]:
## Retriever 3 - Uses a Ranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(base_compressor=reranker_compressor,base_retriever=compressor_retriever)

In [40]:
query = "what is the old capital of India"
respose = final_retriever.invoke(query)
respose

[Document(metadata={'aritcle_id': '4062', 'title': 'Kolkata'}, page_content='started, in 1756 the British began to upgrade their fortifications. When this was protested, the Nawab of Bengal Siraj-Ud-Daulah attacked and captured Fort William. This led to the infamous Black Hole incident. A force of Company sepoys and British troops led by Robert Clive recaptured the city the next year. Calcutta became the capital of British India in 1772. However, the capital shifted to the hilly town of Shimla during the summer months every year, starting from the year 1864. Richard Wellesley, the Governor General between 1797–1805, helped in the growth of the city and its public architecture. This led to the description of Calcutta as "The City of Palaces". The city was a centre of the British East India Company\'s opium trade during the 18th and 19th century; locally produced opium was sold at auction in Kolkata, to be shipped to China.')]

In [41]:
query = "what is teh fastest animal"
response = final_retriever.invoke(query)
response

[Document(metadata={'aritcle_id': '9800', 'title': 'Cheetah'}, page_content='A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.The cheetah compared to other big cats is light and slimly built. Its long thin legs and long spotted tail are necessary for fast running. Its lightly built, thin form is in sharp contrast with the robust build of other big cats. The head-and-body length ranges from . The cheetah stands 70 to 90\xa0cm at the shoulder, and weighs .The slightly curved claws are only weakly retractable (semi-retractable). This is a major point of difference between the cheetah and the other big cats, which have fully retractable claws.')]

## Building a QA RAG chain

In [42]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know.
            Do not make up the answer unless it is there in the provided context.
            Give a detailed answer with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

### LCEL syntax for QA RAG chain

In [45]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
  return '\n\n'.join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (final_retriever
                    |
                    format_docs),
        "question": RunnablePassthrough()
    }
    |
    prompt_template
    |
    chatgpt
)

In [47]:
from IPython.display import Markdown, display

query = "waht is the financial capital of India"
response = qa_rag_chain.invoke(query)
display(Markdown(response.content))

The financial capital of India is Mumbai. Mumbai is not only the largest city in India but also serves as the financial hub of the country. It generates more than 6% of India's GDP and accounts for 25% of industrial output, 40% of sea trade, and 70% of capital to India's economy. The city is home to the Reserve Bank of India, the Bombay Stock Exchange, the National Stock Exchange of India, and many Indian companies and multinational corporations. Mumbai's economic significance and contribution to India's economy make it the financial capital of the country.

In [49]:
query = "waht is the fastest animal?"
response = qa_rag_chain.invoke(query)
display(Markdown(response.content))

The fastest animal is the cheetah. It can run up to 112 kilometers per hour for a short time, making it the fastest land animal. Cheetahs are known for their speed and agility, which they use to hunt their prey in the savannas of Africa. Their long thin legs, long spotted tail, and light, slim build are all adaptations that help them achieve such high speeds. Compared to other big cats, cheetahs are uniquely built for speed with their lightly built, thin form and semi-retractable claws.

In [51]:
query = "tell me more about 2021"
response = qa_rag_chain.invoke(query)
display(Markdown(response.content))

I'm sorry, without any specific context or information provided, I don't have any details to share about the year 2021. If you have any specific questions or topics related to 2021 that you would like to know more about, please feel free to ask.